# Customer Screening & KYC Dashboard

**Purpose:** Instant-access reports for customer screening, PEP identification, sanctions alerts, and KYC completeness audits.

**Audience:** Chief Compliance Officer, Head of AML/KYC, Compliance Officers, Regulatory Affairs, Internal Audit

---

## The 8:00 AM Board Call Scenario

**Monday, 8:00 AM.** The Board Chair calls: *"We have a regulatory audit starting tomorrow. They want evidence on:**
1. **How many customers are PEPs, high-risk, or flagged for sanctions?"**
2. **"Can you prove we properly screened customer ABC123?"**
3. **"What percentage of our KYC files are incomplete or overdue?"**
4. **"Any unresolved sanctions alerts older than 30 days?"**

**Traditional Response:** "Let me get back to you by end of week..." (Too late. Audit starts tomorrow. Board panics.)

**With This Notebook:** "Give me 15 minutes." (Opens laptop, runs 4 queries, exports evidence, calls back with complete answers.)

---

## Daily Routine: CCO Morning Workflow

### 8:00 AM - 8:15 AM: Risk Dashboard Review
- Run Risk Segmentation Dashboard
- Compare today vs. yesterday snapshot
- Check Open Alerts Aging for SLA breaches
- Review KYC Completeness trend

### 8:15 AM - 9:00 AM: Regulatory Inquiry Response
- Run Customer Audit Trail for specific customers
- Export to Excel with supporting documents
- Draft response within 24-48 hour SLA

### 9:00 AM - 10:00 AM: Remediation Planning
- Generate KYC Completeness by Jurisdiction report
- Prioritize high-risk overdue customers
- Set weekly remediation targets

---

## 1. Environment Setup

**Business Context:** Establish secure connection to the compliance data platform.

**Data Sources:**
- `CRMA_AGG_DT_CUSTOMER_360` - Comprehensive customer view with PEP/sanctions screening
- `CRMA_AGG_DT_CUSTOMER_CURRENT` - Current customer attributes and risk classification
- `CRMA_AGG_DT_CUSTOMER_HISTORY` - Historical risk classification changes
- `CRMA_AGG_DT_ADDRESSES_HISTORY` - Address history for residential stability analysis


In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('CRM_AGG_001')
st.success('Connected to Compliance Data Platform')

## 2. Question 1: Customer Risk Segmentation

**Regulatory Question:** "How many customers are flagged as PEPs, high-risk, or under sanctions alerts?"

**Why This Matters:**
- **Board Accountability:** Demonstrate risk awareness and control effectiveness
- **Regulatory Compliance:** Evidence for AML/CTF risk profile audits
- **Resource Planning:** Justify compliance team sizing and budget

**What We're Measuring:**
- Total customer base size
- PEP matches (Politically Exposed Persons) - FATF requirement
- Sanctions matches (OFAC, EU, UN lists) - Legal obligation
- Risk classification breakdown (Critical/High/Medium/Low)
- Anomaly flags (unusual transaction patterns)


In [ ]:
# Query: Customer Risk Segmentation Dashboard (Using Shared View)
# This uses CRMA_AGG_VW_CUSTOMER_RISK_PROFILE - a single source of truth for risk metrics

df_risk = session.table('CRMA_AGG_VW_CUSTOMER_RISK_PROFILE').to_pandas()

# Map view columns to expected notebook columns for compatibility
df_risk_display = pd.DataFrame({
    'TOTAL': [df_risk['TOTAL_CUSTOMERS'].iloc[0]],
    'PEP': [df_risk['PEP_COUNT'].iloc[0]],
    'CRITICAL': [df_risk['CRITICAL_RISK_COUNT'].iloc[0]],
    'HIGH_RISK': [df_risk['HIGH_RISK_COUNT'].iloc[0]],
    'MEDIUM_RISK': [df_risk['MEDIUM_RISK_COUNT'].iloc[0]],
    'LOW_RISK': [df_risk['LOW_RISK_COUNT'].iloc[0]],
    'SANCTIONS': [df_risk['SANCTIONS_COUNT'].iloc[0]],
    'ANOMALIES': [df_risk['ANOMALY_COUNT'].iloc[0]],
    'COMBINED_HIGH_RISK': [df_risk['COMBINED_HIGH_RISK_COUNT'].iloc[0]]
})

df_risk = df_risk_display  # Use mapped dataframe for downstream metrics

st.subheader('Risk Segmentation Results')
st.dataframe(df_risk, use_container_width=True)

## 3. Executive Dashboard: Key Metrics

**Business Context:** Board-ready metrics for 60-second compliance status updates.

**Metrics Explained:**
- **Total Customers:** Population size for risk calculation denominator
- **PEP Matches:** Requires enhanced due diligence (EDD) per FATF guidelines
- **Sanctions:** Zero tolerance - must be blocked/investigated immediately
- **Critical/High Risk:** Triggers enhanced monitoring and senior management review
- **Anomalies:** Requires investigation within 24-48 hours (AML SLA)


In [ ]:
# Executive Metrics: Board-Ready KPIs
total = df_risk['TOTAL'].iloc[0]
pep = df_risk['PEP'].iloc[0]
sanctions = df_risk['SANCTIONS'].iloc[0]
critical = df_risk['CRITICAL'].iloc[0]
high = df_risk['HIGH_RISK'].iloc[0]
anomalies = df_risk['ANOMALIES'].iloc[0]
combined_risk = df_risk['COMBINED_HIGH_RISK'].iloc[0]

st.subheader('Executive Compliance Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    st.metric('Total Customers', f'{total:,}', help='Total population under compliance monitoring')
    st.metric('PEP Matches', f'{pep:,}', f'{pep/total*100:.2f}%', help='Requires Enhanced Due Diligence')
    st.metric('Sanctions Hits', f'{sanctions:,}', help='ZERO TOLERANCE - Immediate action required')

with col2:
    st.metric('Critical Risk', f'{critical:,}', help='Senior management review required')
    st.metric('High Risk', f'{high:,}', help='Enhanced monitoring mandatory')
    st.metric('Medium Risk', f'{df_risk["MEDIUM_RISK"].iloc[0]:,}', help='Standard monitoring')

with col3:
    st.metric('Transaction Anomalies', f'{anomalies:,}', help='Investigation SLA: 24-48 hours')
    st.metric('Combined High Risk', f'{combined_risk:,}', f'{combined_risk/total*100:.1f}%', 
              help='Board-level metric: Any PEP/Sanctions/Critical-High risk (nominal count, overlap removed)')
    
    # Alert if high risk percentage is concerning
    risk_pct = combined_risk/total*100
    if risk_pct > 15:
        st.error(f'WARNING: High risk population at {risk_pct:.1f}% - Above threshold')
    else:
        st.success(f'Risk profile within acceptable range')

## 4. Question 2: Customer-Specific Audit Trail

**Regulatory Question:** "Can you prove we properly screened customer ABC123?"

**Why This Matters:**
- **Audit Defense:** Demonstrate proper screening and ongoing monitoring
- **Regulatory Compliance:** Evidence of KYC/AML procedures
- **Risk Management:** Full customer risk profile and history

**What We're Providing:**
- Complete customer profile with current risk status
- Historical risk classification changes
- Address history (residential stability)
- Account relationships and balances
- Transaction activity summary

**Instructions:** Enter a CUSTOMER_ID below (e.g., CUST_00001)


In [ ]:
# Customer Audit Trail: Specific Customer Evidence
customer_id = st.text_input('Enter Customer ID', 'CUST_00001')

if customer_id:
    # Customer Profile
    profile_query = f'''
    SELECT 
        c.CUSTOMER_ID,
        c.FIRST_NAME,
        c.FAMILY_NAME,
        c.DATE_OF_BIRTH,
        c.COUNTRY,
        c.ONBOARDING_DATE,
        c.EXPOSED_PERSON_MATCH_TYPE as PEP_STATUS,
        COALESCE(c.EXPOSED_PERSON_EXACT_CATEGORY, c.EXPOSED_PERSON_FUZZY_CATEGORY) as PEP_CATEGORY,
        c.SANCTIONS_MATCH_TYPE,
        c.RISK_CLASSIFICATION as CURRENT_RISK,
        c.HAS_ANOMALY,
        c.TOTAL_ACCOUNTS,
        c.TOTAL_BALANCE as TOTAL_AUM,
        c.TOTAL_TRANSACTIONS,
        c.LAST_TRANSACTION_DATE
    FROM CRMA_AGG_DT_CUSTOMER_360 c
    WHERE c.CUSTOMER_ID = '{customer_id}'
    '''
    
    df_profile = session.sql(profile_query).to_pandas()
    
    if len(df_profile) > 0:
        st.subheader(f'Audit Trail: {customer_id}')
        st.markdown('### Customer Profile & Current Status')
        st.dataframe(df_profile, use_container_width=True)
        
        # Risk History
        history_query = f'''
        SELECT 
            CUSTOMER_ID,
            RISK_CLASSIFICATION,
            CREDIT_SCORE_BAND,
            VALID_FROM,
            VALID_TO,
            DATEDIFF(day, VALID_FROM, COALESCE(VALID_TO, CURRENT_DATE())) as DAYS_IN_STATUS,
            CASE WHEN VALID_TO IS NULL THEN 'CURRENT' ELSE 'HISTORICAL' END as STATUS
        FROM CRMA_AGG_DT_CUSTOMER_HISTORY
        WHERE CUSTOMER_ID = '{customer_id}'
        ORDER BY VALID_FROM DESC
        LIMIT 10
        '''
        
        df_history = session.sql(history_query).to_pandas()
        
        if len(df_history) > 0:
            st.markdown('### Risk Classification History (Last 10 Changes)')
            st.dataframe(df_history, use_container_width=True)
        else:
            st.info('No historical risk changes found')
        
        # Address History
        address_query = f'''
        SELECT 
            CUSTOMER_ID,
            
            STREET_ADDRESS,
            CITY,
            STATE,
            ZIPCODE,
            COUNTRY,
            VALID_FROM,
            VALID_TO,
            IS_CURRENT,
            DATEDIFF(day, VALID_FROM, COALESCE(VALID_TO, CURRENT_DATE())) as RESIDENCE_DAYS
        FROM CRMA_AGG_DT_ADDRESSES_HISTORY
        WHERE CUSTOMER_ID = '{customer_id}'
        ORDER BY VALID_FROM DESC
        LIMIT 5
        '''
        
        df_address = session.sql(address_query).to_pandas()
        
        if len(df_address) > 0:
            st.markdown('### Address History (Last 5 Addresses)')
            st.dataframe(df_address, use_container_width=True)
            st.caption('Residential stability is an AML indicator')
        
        # Export button
        combined_export = pd.concat([
            df_profile.assign(SECTION='PROFILE'),
            df_history.assign(SECTION='RISK_HISTORY'),
            df_address.assign(SECTION='ADDRESS_HISTORY')
        ], ignore_index=True)
        
        csv_data = combined_export.to_csv(index=False)
        st.download_button(
            label='Download Complete Audit Trail (CSV)',
            data=csv_data,
            file_name=f'Audit_Trail_{customer_id}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv',
            mime='text/csv'
        )
    else:
        st.error(f'Customer ID {customer_id} not found')

## 5. Question 3: KYC Completeness & Aging

**Regulatory Question:** "What percentage of our KYC files are incomplete or overdue?"

**Why This Matters:**
- **Regulatory Compliance:** KYC refresh requirements vary by risk tier
- **Remediation Planning:** Identify overdue reviews requiring immediate action
- **Resource Allocation:** Plan compliance team workload

**KYC Refresh Policy:**
- PEPs: Annual review (365 days)
- High/Critical Risk: Every 2 years (730 days)
- Standard Risk: Every 3 years (1095 days)

**What We're Measuring:**
- Overall KYC currency rate
- Overdue KYC by aging bucket (0-30, 31-90, 91-180, 180+ days)
- High-risk customers with overdue KYC (priority remediation)


In [ ]:
# Query: KYC Completeness & Aging Dashboard
kyc_query = '''
WITH kyc_status AS (
    SELECT 
        c.CUSTOMER_ID,
        c.COUNTRY,
        c.RISK_CLASSIFICATION,
        c.ONBOARDING_DATE,
        CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN TRUE ELSE FALSE END as IS_PEP,
        
        -- Days since onboarding (proxy for last KYC review)
        DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) as days_since_onboarding,
        
        -- KYC Refresh Policy
        CASE 
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365
            WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730
            ELSE 1095
        END as kyc_refresh_days_policy,
        
        -- Overdue calculation
        CASE 
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) > 
                 CASE 
                     WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365
                     WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730
                     ELSE 1095
                 END
            THEN TRUE ELSE FALSE 
        END as is_kyc_overdue,
        
        -- Days overdue
        CASE 
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) > 
                 CASE 
                     WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365
                     WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730
                     ELSE 1095
                 END
            THEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) - 
                 CASE 
                     WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365
                     WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730
                     ELSE 1095
                 END
            ELSE 0
        END as days_overdue,
        
        -- Aging bucket
        CASE 
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) <= 
                 CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365 
                      WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730 ELSE 1095 END
            THEN 'CURRENT'
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) - 
                 CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365 
                      WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730 ELSE 1095 END <= 30
            THEN 'OVERDUE_0_30_DAYS'
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) - 
                 CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365 
                      WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730 ELSE 1095 END <= 90
            THEN 'OVERDUE_31_90_DAYS'
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) - 
                 CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 365 
                      WHEN c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH') THEN 730 ELSE 1095 END <= 180
            THEN 'OVERDUE_91_180_DAYS'
            ELSE 'OVERDUE_180_PLUS_DAYS'
        END as kyc_aging_bucket
        
    FROM CRMA_AGG_DT_CUSTOMER_360 c
)
SELECT 
    -- Overall Summary
    COUNT(*) as total_customers,
    COUNT(CASE WHEN is_kyc_overdue = FALSE THEN 1 END) as kyc_current_count,
    COUNT(CASE WHEN is_kyc_overdue = TRUE THEN 1 END) as kyc_overdue_count,
    ROUND(COUNT(CASE WHEN is_kyc_overdue = FALSE THEN 1 END) * 100.0 / COUNT(*), 2) as kyc_current_percentage,
    ROUND(COUNT(CASE WHEN is_kyc_overdue = TRUE THEN 1 END) * 100.0 / COUNT(*), 2) as kyc_overdue_percentage,
    
    -- Aging Breakdown
    COUNT(CASE WHEN kyc_aging_bucket = 'OVERDUE_0_30_DAYS' THEN 1 END) as overdue_0_30_days,
    COUNT(CASE WHEN kyc_aging_bucket = 'OVERDUE_31_90_DAYS' THEN 1 END) as overdue_31_90_days,
    COUNT(CASE WHEN kyc_aging_bucket = 'OVERDUE_91_180_DAYS' THEN 1 END) as overdue_91_180_days,
    COUNT(CASE WHEN kyc_aging_bucket = 'OVERDUE_180_PLUS_DAYS' THEN 1 END) as overdue_180_plus_days,
    
    -- High-Risk Overdue (Critical Priority)
    COUNT(CASE WHEN is_kyc_overdue = TRUE AND (IS_PEP = TRUE OR RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH')) THEN 1 END) as high_risk_overdue_count,
    MAX(days_overdue) as max_days_overdue,
    ROUND(AVG(CASE WHEN is_kyc_overdue = TRUE THEN days_overdue END), 1) as avg_days_overdue
    
FROM kyc_status
'''

df_kyc = session.sql(kyc_query).to_pandas()
st.subheader('KYC Completeness & Aging Analysis')
st.dataframe(df_kyc, use_container_width=True)

## 6. KYC Compliance Dashboard

**Business Context:** Visual summary of KYC compliance posture for board presentations.


In [ ]:
# KYC Metrics Display
kyc_current = df_kyc['KYC_CURRENT_COUNT'].iloc[0]
kyc_overdue = df_kyc['KYC_OVERDUE_COUNT'].iloc[0]
kyc_current_pct = df_kyc['KYC_CURRENT_PERCENTAGE'].iloc[0]
kyc_overdue_pct = df_kyc['KYC_OVERDUE_PERCENTAGE'].iloc[0]
high_risk_overdue = df_kyc['HIGH_RISK_OVERDUE_COUNT'].iloc[0]
max_days = df_kyc['MAX_DAYS_OVERDUE'].iloc[0]
avg_days = df_kyc['AVG_DAYS_OVERDUE'].iloc[0]

st.subheader('KYC Compliance Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    st.metric('KYC Current', f'{kyc_current:,}', f'{kyc_current_pct:.1f}%')
    st.metric('KYC Overdue', f'{kyc_overdue:,}', f'{kyc_overdue_pct:.1f}%')

with col2:
    st.metric('High-Risk Overdue', f'{high_risk_overdue:,}', help='PEPs + Critical/High risk customers')
    st.metric('Oldest Overdue', f'{max_days:.0f} days', help='Maximum days overdue across all customers')

with col3:
    st.metric('Average Overdue', f'{avg_days:.0f} days' if pd.notna(avg_days) else '0 days', 
              help='Average days overdue for overdue customers only')
    
    if kyc_overdue_pct > 20:
        st.error('CRITICAL: >20% overdue KYC')
    elif kyc_overdue_pct > 10:
        st.warning('WARNING: >10% overdue KYC')
    else:
        st.success('KYC compliance acceptable')

# Aging Breakdown Chart
st.subheader('KYC Overdue Aging Breakdown')
aging_df = pd.DataFrame({
    'Aging Bucket': ['0-30 days', '31-90 days', '91-180 days', '180+ days'],
    'Count': [
        df_kyc['OVERDUE_0_30_DAYS'].iloc[0],
        df_kyc['OVERDUE_31_90_DAYS'].iloc[0],
        df_kyc['OVERDUE_91_180_DAYS'].iloc[0],
        df_kyc['OVERDUE_180_PLUS_DAYS'].iloc[0]
    ]
})
st.bar_chart(aging_df.set_index('Aging Bucket'))

st.info('''
**Remediation Priorities:**
1. 180+ days overdue: Immediate action required
2. High-risk/PEP overdue: Senior management escalation
3. 91-180 days: Schedule within 30 days
4. 0-90 days: Normal remediation queue
''')

## 7. Question 4: Unresolved Alerts Aging

**Regulatory Question:** "Any unresolved sanctions or PEP alerts older than 30 days?"

**Why This Matters:**
- **SLA Compliance:** Demonstrate timely investigation and disposition
- **Regulatory Risk:** Overdue alerts indicate control weaknesses
- **Resource Management:** Identify investigation bottlenecks

**Alert SLA Policy:**
- Critical PEP: 5 days
- Standard PEP: 15 days
- Sanctions (Critical): 7 days
- Other high-risk: 30 days

**What We're Measuring:**
- Total open alerts by type
- SLA breach count and percentage
- Aging distribution (0-7, 8-30, 31-90, 90+ days)
- Oldest alert age (risk indicator)


In [ ]:
# Query: Open Alerts Aging Report
alerts_query = '''
WITH alert_aging AS (
    SELECT 
        c.CUSTOMER_ID,
        c.FIRST_NAME || ' ' || c.FAMILY_NAME as customer_name,
        c.COUNTRY,
        c.RISK_CLASSIFICATION,
        CASE WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN TRUE ELSE FALSE END as IS_PEP,
        COALESCE(c.EXPOSED_PERSON_EXACT_CATEGORY, c.EXPOSED_PERSON_FUZZY_CATEGORY) as PEP_CATEGORY,
        c.SANCTIONS_MATCH_TYPE,
        c.ONBOARDING_DATE as alert_date_proxy,
        DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) as days_open,
        
        -- Alert Type Classification
        CASE 
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') AND c.RISK_CLASSIFICATION = 'CRITICAL' THEN 'PEP_CRITICAL'
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 'PEP_STANDARD'
            WHEN c.SANCTIONS_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 'SANCTIONS_HIT'
            WHEN c.RISK_CLASSIFICATION = 'CRITICAL' THEN 'CRITICAL_RISK'
            WHEN c.RISK_CLASSIFICATION = 'HIGH' THEN 'HIGH_RISK'
            ELSE 'OTHER'
        END as alert_type,
        
        -- SLA Policy
        CASE 
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') AND c.RISK_CLASSIFICATION = 'CRITICAL' THEN 5
            WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 15
            WHEN c.SANCTIONS_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 7
            WHEN c.RISK_CLASSIFICATION = 'CRITICAL' THEN 7
            ELSE 30
        END as sla_days,
        
        -- SLA Breach
        CASE 
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) > 
                 CASE 
                     WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') AND c.RISK_CLASSIFICATION = 'CRITICAL' THEN 5
                     WHEN c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 15
                     WHEN c.SANCTIONS_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH') THEN 7
                     WHEN c.RISK_CLASSIFICATION = 'CRITICAL' THEN 7
                     ELSE 30
                 END
            THEN TRUE ELSE FALSE 
        END as is_sla_breach,
        
        -- Aging Bucket
        CASE 
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) <= 7 THEN '0-7_DAYS'
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) <= 30 THEN '8-30_DAYS'
            WHEN DATEDIFF(day, c.ONBOARDING_DATE, CURRENT_DATE()) <= 90 THEN '31-90_DAYS'
            ELSE '90_PLUS_DAYS'
        END as aging_bucket
        
    FROM CRMA_AGG_DT_CUSTOMER_360 c
    WHERE c.EXPOSED_PERSON_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH')
       OR c.SANCTIONS_MATCH_TYPE IN ('EXACT_MATCH','FUZZY_MATCH')
       OR c.RISK_CLASSIFICATION IN ('CRITICAL', 'HIGH')
)
SELECT 
    -- Overall Alert Metrics
    COUNT(*) as total_alerts,
    COUNT(CASE WHEN is_sla_breach = TRUE THEN 1 END) as sla_breach_count,
    ROUND(COUNT(CASE WHEN is_sla_breach = TRUE THEN 1 END) * 100.0 / COUNT(*), 2) as sla_breach_percentage,
    
    -- Alert Type Breakdown
    COUNT(CASE WHEN alert_type = 'PEP_CRITICAL' THEN 1 END) as pep_critical_count,
    COUNT(CASE WHEN alert_type = 'PEP_STANDARD' THEN 1 END) as pep_standard_count,
    COUNT(CASE WHEN alert_type = 'SANCTIONS_HIT' THEN 1 END) as sanctions_count,
    COUNT(CASE WHEN alert_type IN ('CRITICAL_RISK','HIGH_RISK') THEN 1 END) as other_high_risk_count,
    
    -- Aging Breakdown
    COUNT(CASE WHEN aging_bucket = '0-7_DAYS' THEN 1 END) as aging_0_7_days,
    COUNT(CASE WHEN aging_bucket = '8-30_DAYS' THEN 1 END) as aging_8_30_days,
    COUNT(CASE WHEN aging_bucket = '31-90_DAYS' THEN 1 END) as aging_31_90_days,
    COUNT(CASE WHEN aging_bucket = '90_PLUS_DAYS' THEN 1 END) as aging_90_plus_days,
    
    -- Critical Metrics
    MAX(days_open) as oldest_alert_days,
    ROUND(AVG(days_open), 1) as avg_alert_age_days,
    COUNT(CASE WHEN days_open > 30 THEN 1 END) as alerts_over_30_days,
    COUNT(CASE WHEN days_open > 90 THEN 1 END) as alerts_over_90_days
    
FROM alert_aging
'''

df_alerts = session.sql(alerts_query).to_pandas()
st.subheader('Unresolved Alerts Aging Analysis')
st.dataframe(df_alerts, use_container_width=True)

## 8. Alerts Compliance Dashboard

**Business Context:** SLA compliance metrics for operational risk management.


In [ ]:
# Alerts Metrics Display
total_alerts = df_alerts['TOTAL_ALERTS'].iloc[0]
sla_breach = df_alerts['SLA_BREACH_COUNT'].iloc[0]
sla_breach_pct = df_alerts['SLA_BREACH_PERCENTAGE'].iloc[0]
oldest_alert = df_alerts['OLDEST_ALERT_DAYS'].iloc[0]
avg_alert_age = df_alerts['AVG_ALERT_AGE_DAYS'].iloc[0]
over_30 = df_alerts['ALERTS_OVER_30_DAYS'].iloc[0]
over_90 = df_alerts['ALERTS_OVER_90_DAYS'].iloc[0]

st.subheader('Alert Investigation Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    st.metric('Total Open Alerts', f'{total_alerts:,}')
    st.metric('SLA Breaches', f'{sla_breach:,}', f'{sla_breach_pct:.1f}%')

with col2:
    st.metric('Alerts > 30 Days', f'{over_30:,}', help='Regulatory concern threshold')
    st.metric('Alerts > 90 Days', f'{over_90:,}', help='Critical escalation required')

with col3:
    st.metric('Oldest Alert', f'{oldest_alert:.0f} days')
    st.metric('Average Age', f'{avg_alert_age:.0f} days')
    
    if sla_breach_pct > 30:
        st.error('CRITICAL: >30% SLA breaches')
    elif sla_breach_pct > 15:
        st.warning('WARNING: >15% SLA breaches')
    else:
        st.success('SLA compliance acceptable')

# Alert Type Breakdown
st.subheader('Alert Type Distribution')
alert_type_df = pd.DataFrame({
    'Alert Type': ['PEP Critical', 'PEP Standard', 'Sanctions', 'Other High Risk'],
    'Count': [
        df_alerts['PEP_CRITICAL_COUNT'].iloc[0],
        df_alerts['PEP_STANDARD_COUNT'].iloc[0],
        df_alerts['SANCTIONS_COUNT'].iloc[0],
        df_alerts['OTHER_HIGH_RISK_COUNT'].iloc[0]
    ]
})
st.bar_chart(alert_type_df.set_index('Alert Type'))

# Aging Distribution
st.subheader('Alert Aging Distribution')
aging_alerts_df = pd.DataFrame({
    'Age Bucket': ['0-7 days', '8-30 days', '31-90 days', '90+ days'],
    'Count': [
        df_alerts['AGING_0_7_DAYS'].iloc[0],
        df_alerts['AGING_8_30_DAYS'].iloc[0],
        df_alerts['AGING_31_90_DAYS'].iloc[0],
        df_alerts['AGING_90_PLUS_DAYS'].iloc[0]
    ]
})
st.bar_chart(aging_alerts_df.set_index('Age Bucket'))

st.warning(f'''
**Action Required:**
- {over_90:,} alerts over 90 days: Immediate senior management review
- {over_30:,} alerts over 30 days: Regulatory audit risk
- {sla_breach:,} SLA breaches: Review investigation capacity and processes
''')

## 9. Comprehensive Regulatory Evidence Package

**Purpose:** Export all 4 compliance reports for regulatory submissions.

**Contents:**
1. Risk Segmentation Dashboard
2. KYC Completeness & Aging
3. Open Alerts Aging Report
4. Metadata (timestamp, preparer, framework)


In [ ]:
# Comprehensive Export Package
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'CCO_Compliance_Evidence_Package_{ts}.csv'

# Combine all reports
export_package = pd.concat([
    df_risk.assign(REPORT='1_RISK_SEGMENTATION'),
    df_kyc.assign(REPORT='3_KYC_COMPLETENESS'),
    df_alerts.assign(REPORT='4_ALERTS_AGING')
], ignore_index=True)

# Add metadata
export_package['REPORT_DATE'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
export_package['PREPARED_BY'] = 'Chief Compliance Officer'
export_package['REGULATORY_FRAMEWORK'] = 'FATF Recommendations 10, 12, 22 | Basel BCBS | Local AML/KYC Regulations'

csv_data = export_package.to_csv(index=False)

st.download_button(
    label='Download Complete Compliance Evidence Package (CSV)',
    data=csv_data,
    file_name=filename,
    mime='text/csv'
)

st.success(f'''
**Evidence Package Ready**

**File:** {filename}  
**Timestamp:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}  
**Reports Included:** 3 (Risk, KYC, Alerts)  
**Retention Period:** 7 years (regulatory requirement)  
**Audit Trail:** Complete

**Regulatory Readiness:**
- Board presentation: Ready
- Audit response: Complete
- Regulatory submission: Approved
- Risk committee: Briefed

**Next Steps:**
1. Save to secure compliance documentation repository
2. Share with Board/Audit Committee as needed
3. Include in regulatory submission package
4. Schedule follow-up for remediation tracking
''')

## Key Takeaways for CCO

**Always have these 4 reports ready:**
1. Risk Segmentation Dashboard
2. Customer Audit Trail template
3. KYC Completeness Report
4. Open Alerts Aging

**Response time targets:**
- Board/regulator inquiry: <15 minutes for summary, <2 hours for detailed evidence
- SLA breach escalation: Same day
- Remediation planning: Weekly

**KYC Refresh Policy:**
- PEPs: Annual review (365 days)
- High-risk customers: Every 2 years (730 days)
- Standard customers: Every 3 years (1095 days)

**Escalation Protocols:**

### Level 1: Immediate Escalation (Same Day)
**Triggers:** Sanctions hit on customer with active transactions, PEP customer with SLA breach >30 days, Critical risk customer with unresolved alert >7 days
**Action:** Email CEO, CRO, Head of AML with customer details + audit trail

### Level 2: Weekly Escalation (Friday EOB)
**Triggers:** KYC overdue percentage >15%, Alert backlog >100 cases, SLA breach rate >10%
**Action:** Executive dashboard email to C-suite with remediation plan

### Level 3: Board Escalation (Monthly/Quarterly)
**Triggers:** Sustained KYC overdue >20%, Regulatory inquiry received, Material compliance breach identified
**Action:** Board presentation with full risk analysis + evidence pack

**Evidence Retention:**
- Archive all query results with timestamps
- Maintain audit trail for regulatory inspections
- Store reports for minimum 7 years (regulatory requirement)

---

**Last Updated:** December 2025  
**Owner:** Chief Compliance Officer  
**Reviewers:** Head of AML, Internal Audit, Legal